In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.extmath import randomized_svd
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
df = pd.read_csv('./movies.csv',encoding='utf-8')

### Funcion pelicula_mes

In [21]:
# Primera funcion
def peliculas_mes(mes):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''
    mes = mes.lower().strip()
    cantidad = len(df.loc[df['month'] == mes, 'title'])
    return {'mes':mes,'cantidad':cantidad}


In [22]:
peliculas_mes('enero')

{'mes': 'enero', 'cantidad': 5912}

### Funcion peliculas_dia

In [23]:
def peliculas_dia(dia):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente'''
    dia = dia.lower().strip()
    cantidad = len(df.loc[df['day'] == dia, 'title'])
    return {'dia':dia, 'cantidad':cantidad}

In [24]:
peliculas_dia('lunes')

{'dia': 'lunes', 'cantidad': 3503}

### Funcion franquicia

In [25]:
def franquicia(franquicia):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''
    franquicia = franquicia.lower().strip()
    cantidad = len(df.loc[df["belongs_to_collection"] == franquicia])
    ganancia_total = df["revenue"].loc[df["belongs_to_collection"] == franquicia].sum()
    ganancia_promedio = ganancia_total/cantidad
    return {'franquicia':franquicia, 'cantidad':cantidad, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}

In [26]:
franquicia('toy story collection')

{'franquicia': 'toy story collection',
 'cantidad': 3,
 'ganancia_total': 1937890605.0,
 'ganancia_promedio': 645963535.0}

### Funcion peliculas_pais

In [27]:
def peliculas_pais(pais): 
    pais = pais.title()
    cantidad = 0
    lista = df["production_countries"]
    for i in range(len(lista)):
        if lista[i] is None:
            continue
        if type(lista[i]) is not list and lista[i] == pais:
            cantidad += 1
        if pais in lista[i]:
            cantidad += 1
    return {'pais':pais, 'cantidad':cantidad}

In [28]:
peliculas_pais('United Kingdom')

{'pais': 'United Kingdom', 'cantidad': 4091}

### Funcion productoras

In [29]:
def productoras(productora:str):
    '''Ingresas la productora, retornando la ganancia toal y la cantidad de peliculas que produjeron'''
    productora = productora.title()
    ganancia_total = 0
    cantidad = 0
    lista = df["production_companies"]
    for i in range(len(lista)):
        if lista[i] is None:
            continue
        if type(lista[i]) is not list and lista[i] == productora:
            cantidad += 1
            ganancia = df["revenue"][i]
            ganancia_total += ganancia
        if productora in lista[i]:
            cantidad += 1
            ganancia = df["revenue"][i]
            ganancia_total += ganancia
    
    
    return {'productora': productora, 'ganancia_total': ganancia_total, 'cantidad': cantidad}


In [30]:
productoras('Pixar Animation Studios')

{'productora': 'Pixar Animation Studios',
 'ganancia_total': 11188533734.0,
 'cantidad': 52}

### Funcion retorno

In [31]:
def retorno(pelicula):
    '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo'''
    pelicula = pelicula.title()
    inversion = sum(df["budget"][df["title"] == pelicula])
    ganancia = sum(df["revenue"][df["title"] == pelicula])
    retorno_dos = sum(df["return"][df["title"] == pelicula])
    year = df["release_year"][df["title"] == pelicula].item()
    return {'pelicula':pelicula,'inversion':inversion,'ganancia':ganancia,'retorno':retorno_dos,'anio':year}

In [32]:
retorno('toy story')

{'pelicula': 'Toy Story',
 'inversion': 30000000,
 'ganancia': 373554033.0,
 'retorno': 12.4518011,
 'anio': 1995}

### Funcion recomendacion

In [33]:
# El Vectorizador TfidfVectorizer con parámetros de reduccion procesamiento
df['genres'].fillna('', inplace=True)
vectorizar = TfidfVectorizer(min_df=10, max_df=0.5, ngram_range=(1,2))

# Vectorizamos, ajustamos y transformamos el texto de la columna "title" del DataFrame
X = vectorizar.fit_transform(df['genres'])

# Calcular la matriz de similitud de coseno con una matriz reducida de 7000
similarity_matrix = cosine_similarity(X[:1250,:])

# Obtenemos la descomposición en valores singulares aleatoria de la matriz de similitud de coseno con 10 componentes
n_components = 10
U, Sigma, VT = randomized_svd(similarity_matrix, n_components=n_components)

# Construir la matriz reducida de similitud de coseno
reduced_similarity_matrix = U.dot(np.diag(Sigma)).dot(VT)

In [34]:
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    titulo = titulo.title()
    #Ubicamos el indice del titulo pasado como parametro en la columna 'title' del dts user_item
    indice = np.where(df['title'] == titulo)[0][0]
    #Vemos los indices de aquellas puntuaciones y caracteristicas similares hacia el titulo 
    puntuaciones_similitud = reduced_similarity_matrix[indice,:]
    # Se ordena los indicies de menor a mayor
    puntuacion_ordenada = np.argsort(puntuaciones_similitud)[::-1]
    # Que solo 5 nos indique 
    top_indices = puntuacion_ordenada[:5]
    
    return df.loc[top_indices, 'title'].tolist()

In [35]:
recomendacion('toy story')

['Little Big League',
 'Big Bully',
 'Ri¢hie Ri¢h',
 'The Air Up There',
 'Life With Mikey']